In [55]:
import cv2
import pandas as pd
import numpy as np

In [56]:
image = cv2.imread('./test.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))

In [57]:
def generate_window_points(width, height, window_size):
    width_1 = width - window_size + 1
    height_1 = height - window_size + 1
    points = []
    if(width_1 < 0 or height_1 < 0):
        return None
    for i in range(width_1):
        for j in range(height_1):
            points += [[i, j]]
    points = np.array(np.array(points))
    return points
def generate_window(img,x, y, window_size):
    '''
    x is the row index
    y is the column index
    '''
    img_new = []
    for i in range(x, x+window_size):
        row = []
        for j in range(y, y+window_size):
            row += [img[i][j]]
        img_new += [row]
    img_new = np.array(np.array(img_new))
    return img_new


def windows(img, min_wind, ratio):
    '''
    it is assumed that the image of a grayscale one.
    '''
    sizes = []
    temp = min_wind
    while(temp < max(img.shape)):
        sizes += [temp]
        temp *= ratio
    return sizes

def gen_and_resize(sizes, windows_list, img, resize_shape):
    '''
    resize_shape -> tuple
    '''
    windows = []
    total_windows = 0
    for i in windows_list:
        total_windows += len(i)
    done = 0
    for i in range(len(sizes)):
        size_now = sizes[i]
        windows_list_now = windows_list[i]
        windows += [cv2.resize(generate_window(img, j[0], j[1], size_now), resize_shape) for j in windows_list_now]
        done += len(windows_list_now)
        print(f'{(done/total_windows)*100} % done')
    return windows
def return_all_windows(img, window_size, resize_shape, ratio):
    sizes = windows(image, window_size, ratio)
    width = image.shape[0]
    height = image.shape[1]
    windows_list = [generate_window_points(width, height, i) for i in sizes]
    windows_new = gen_and_resize(sizes, windows_list, image, resize_shape)
    return windows_new


In [58]:
window_size = 19
resize_shape = (19, 19)
ratio = 2

In [59]:
windows_new = return_all_windows(image, window_size, resize_shape, ratio)

59.40978971549744 % done
94.47782293691465 % done
100.0 % done
